In [1]:
# Bibliotecas
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [2]:
# Dados
Cidades = ['SC', 'FR', 'RP', 'SJRP', 'PC', 'LI']
# Parâmetros
distancia_i_j = {('SC', 'FR'): 186, ('SC', 'RP'): 105, ('SC', 'SJRP'): 208, ('SC', 'PC'): 177, ('SC', 'LI'): 94, 
                 ('FR', 'SC'): 187, ('FR', 'RP'): 89.8, ('FR', 'SJRP'): 223, ('FR', 'PC'): 255, ('FR', 'LI'): 254, 
                 ('RP', 'SC'): 99.9, ('RP', 'FR'): 89, ('RP', 'SJRP'): 203, ('RP', 'PC'): 204, ('RP', 'LI'): 172, 
                 ('SJRP', 'FR'): 220, ('SJRP', 'RP'): 203, ('SJRP', 'SC'): 206, ('SJRP', 'PC'): 377, ('SJRP', 'LI'): 295, 
                 ('PC', 'FR'): 251, ('PC', 'RP'): 201, ('PC', 'SJRP'): 376, ('PC', 'SC'): 168, ('PC', 'LI'): 156, 
                 ('LI', 'FR'): 255, ('LI', 'RP'): 173, ('LI', 'SJRP'): 293, ('LI', 'PC'): 159, ('LI', 'SC'): 86.8}

In [3]:
# Modelo
model = ConcreteModel()

# Conjuntos
model.I = Set(initialize=Cidades)
I = model.I

# Parâmetros
model.D = Param(I * I, initialize=distancia_i_j, within=NonNegativeReals)
D = model.D

# Variáveis de decisão
model.x = Var(I * I, within=Binary)
x = model.x

# Função objetivo
model.obj = Objective(expr=sum(sum(D[i, j] * x[i, j] for j in I if i != j) for i in I))

# Restrições
# Uma cidade visitada por vez
model.r_visitada = ConstraintList()
for i in I:
    model.r_visitada.add(expr=sum(x[i, j] for j in I if j != i) == 1)
# Cada cidade tem uma única antecessora
model.r_antecessora = ConstraintList()
for j in I:
    model.r_antecessora.add(expr=sum(x[i, j] for i in I if i != j) == 1)
# Remover sub-rota SC-SJRP-SC
model.r_sub_1 = ConstraintList()
model.r_sub_1.add(expr=x['SC', 'SJRP'] + x['SJRP', 'SC'] <= 1)
# Remover sub-rota SC-PC-LI-SC
model.r_sub_2 = ConstraintList()
model.r_sub_2.add(expr=x['SC', 'PC'] + x['SC', 'LI'] + x['LI', 'PC'] + 
                  x['LI', 'SC'] + x['PC', 'SC'] + x['PC', 'LI'] <= 2 )
# Remover sub-rota PC-LI-PC
model.r_sub_3 = ConstraintList()
model.r_sub_3.add(expr=x['PC', 'LI'] + x['LI', 'PC'] <= 1)

# model.pprint()

In [4]:
# Resolução
solver = SolverFactory('glpk', executable='/usr/bin/glpsol')
resultado = solver.solve(model, tee = False)

In [5]:
# Apresentação de resultados
if (resultado.solver.status == SolverStatus.ok) and (resultado.solver.termination_condition != TerminationCondition.infeasible):
    for i in I:
        for j in I:
            if i != j:
                if value(x[i, j]) == 1:
                    print(x[i, j])
    print()
    print('Valor ótimo: {:.2f}'.format(value(model.obj)))
else:
    print(resultado.solver.termination_condition)

x[SC,SJRP]
x[FR,RP]
x[RP,PC]
x[SJRP,FR]
x[PC,LI]
x[LI,SC]

Valor ótimo: 964.60
